In [1]:
import sys 
sys.path.append("../")

In [2]:
import pandas as pd 
import plotly.graph_objects as go 
from technical.indicators import rsi 
from technical.patterns import apply_patterns
from plotting import CandlePlot

In [3]:
df_raw = pd.read_pickle("../data/GBP_JPY_H1.pkl")

In [4]:
df_raw.shape

(59627, 14)

In [5]:
df_an = df_raw.iloc[-6000:].copy()
df_an.reset_index(drop=True, inplace=True)

In [6]:
df_an.shape

(6000, 14)

In [7]:
df_an = rsi(df_an)
df_an.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,RSI_14
0,2024-01-14 22:00:00+00:00,1667,184.790,184.823,184.673,184.818,184.759,184.759,184.499,184.748,184.820,184.983,184.762,184.889,NaN
1,2024-01-14 23:00:00+00:00,2980,184.820,184.897,184.693,184.863,184.751,184.879,184.678,184.846,184.889,184.915,184.707,184.880,NaN
2,2024-01-15 00:00:00+00:00,7133,184.861,184.900,184.704,184.792,184.844,184.883,184.686,184.775,184.878,184.918,184.722,184.810,NaN
3,2024-01-15 01:00:00+00:00,7577,184.794,184.926,184.732,184.854,184.776,184.908,184.714,184.837,184.812,184.943,184.749,184.872,NaN
4,2024-01-15 02:00:00+00:00,6662,184.858,185.020,184.844,185.006,184.839,185.006,184.826,184.992,184.876,185.038,184.862,185.019,NaN


In [8]:
df_an = apply_patterns(df_an)

In [9]:
df_an["EMA_200"] = df_an.mid_c.ewm(span=200, min_periods=200).mean()

In [10]:
df_an.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', 'RSI_14',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
       'body_size_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev',
       'body_perc_prev_2', 'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR', 'EMA_200'],
      dtype='object')

In [11]:
our_cols= ["time",'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_c', 'bid_c', "ENGULFING", 'direction', 'EMA_200', "RSI_14"]

In [12]:
df_slim = df_an[our_cols].copy()
df_slim.dropna(inplace=True)
df_slim.reset_index(drop=True, inplace=True)

In [13]:
df_slim.head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14
0,2024-01-25 05:00:00+00:00,187.820,187.912,187.746,187.860,187.877,187.844,False,1,187.653036,52.636935
1,2024-01-25 06:00:00+00:00,187.860,188.007,187.812,187.967,187.982,187.952,False,1,187.656643,55.980600
2,2024-01-25 07:00:00+00:00,187.968,188.028,187.674,188.016,188.031,188.000,False,1,187.660766,57.461620
3,2024-01-25 08:00:00+00:00,188.018,188.195,187.926,187.933,187.948,187.918,True,-1,187.663884,54.138906
4,2024-01-25 09:00:00+00:00,187.932,188.040,187.832,187.894,187.910,187.878,False,-1,187.666516,52.599794


In [14]:
BUY = 1 
SELL = -1 
NONE = 0
RSI_LIMIT = 50.0 

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY 
        if row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE
    

In [19]:
df_slim["SIGNAL"] = df_slim.apply(apply_signal, axis=1)

In [20]:
df_slim["SIGNAL"].value_counts()

SIGNAL
 0    5273
 1     360
-1     168
Name: count, dtype: int64

In [21]:
LOSS_FACTOR = -1.0 
PROFIT_FACTOR = 1.5 

def apply_take_profit(row):
    if row.SIGNAL != NONE: 
        return(row.mid_c - row.mid_o) * PROFIT_FACTOR + row.mid_c
    else: 
        return 0.0 
    
def apply_stop_loss(row):
    if row.SIGNAL != NONE:
        return row.mid_o
    else: 
        return 0.0

In [22]:
df_slim["TP"] = df_slim.apply(apply_take_profit, axis=1)
df_slim["SL"] = df_slim.apply(apply_stop_loss, axis=1)

In [24]:
df_slim[df_slim.SIGNAL==SELL].head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14,SINGAL,SIGNAL,TP,SL
67,2024-01-30 00:00:00+00:00,187.492,187.515,187.186,187.336,187.352,187.319,True,-1,187.727701,36.802772,-1,-1,187.1020,187.492
73,2024-01-30 06:00:00+00:00,187.266,187.296,187.062,187.102,187.121,187.084,True,-1,187.698531,33.729158,-1,-1,186.8560,187.266
76,2024-01-30 09:00:00+00:00,187.076,187.154,186.686,186.907,186.921,186.893,True,-1,187.675459,32.876301,-1,-1,186.6535,187.076
81,2024-01-30 14:00:00+00:00,186.986,187.032,186.516,186.616,186.680,186.552,True,-1,187.630328,32.033646,-1,-1,186.0610,186.986
90,2024-01-30 23:00:00+00:00,187.463,187.472,187.043,187.043,187.060,187.026,True,-1,187.605885,43.304690,-1,-1,186.4130,187.463
